In [1]:
rm (list = ls())

# Load libraries needed for general use
library(dplyr)
library(tidyr)
library(ggplot2)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
############### This part is to load biz data from Toronto Open Data Source######################
### The business data is download from https://open.toronto.ca/dataset/municipal-licensing-and-standards-business-licences-and-permits/
### Using code from developer section as follows.
# Load libraries needed for loading data from toronto open data source
if (!require(opendatatoronto)) install.packages("opendatatoronto")
library(opendatatoronto)

# get package
package <- show_package("57b2285f-4f80-45fb-ae3e-41a02c3a137f")
package

# get all resources for this package
resources <- list_package_resources("57b2285f-4f80-45fb-ae3e-41a02c3a137f")

# identify datastore resources; by default, Toronto Open Data sets datastore resource format to CSV for non-geospatial and GeoJSON for geospatial resources
datastore_resources <- filter(resources, tolower(format) %in% c('csv', 'geojson'))

# load the first datastore resource as a sample
biz_data <- filter(datastore_resources, row_number()==1) %>% get_resource()
biz_data

### backup data to biz_data_bak
biz_data_bak = biz_data

###Meaning of the data according to the readme from website are as follows
###Category=Type of licence or permit
###Licence No=Number. of licence issued by City of Toronto
###Operating Name=Name that company operates under
###Issued=Date of issue of licence/permit
###Client Name=Name that the licence is issued to
###Business Phone=Client Business Phone Number
###Business Phone Ext.=Client Business Phone Extension Number
###Licence Address Line 1=First line of client's business address
###Licence Address Line 2=Client address town and province
###Licence Address Line 3=Client address postal code
###Conditions=Restriction on the licence/permit recorded in the licensing system
###Free Form Conditions Line 1=Restriction/comment on the licence/permit
###Free Form Conditions Line 2=Continuation of line 1 
###Plate No.=Licence identifying plate, issued to most vehicles
###Endorsements=Activity permitted under the licence
###Cancel Date=Date the license or permit was cancelled
############### End Of Data loading from  Toronto Open Data Source###############################

Loading required package: opendatatoronto
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'opendatatoronto'"also installing the dependencies 'triebeard', 'proxy', 'urltools', 'httpcode', 'e1071', 'wk', 'crul', 'classInt', 's2', 'units', 'ckanr', 'sf'

Warning message:
"unable to access index for repository https://cran.r-project.org/bin/windows/contrib/3.6:
  cannot open URL 'https://cran.r-project.org/bin/windows/contrib/3.6/PACKAGES'"Packages which are only available in source form, and may need
  compilation of C/C++/Fortran: 'triebeard' 'proxy' 'urltools' 'e1071'
  'wk' 'classInt' 's2' 'units' 'sf'


  These will not be installed


installing the source packages 'httpcode', 'crul', 'ckanr', 'opendatatoronto'

Warning message in install.packages("opendatatoronto"):
"installation of package 'crul' had non-zero exit status"Warning message in install.packages("opendatatoronto"):
"installation of package 'ckanr' had non-zero exit status"Warning message in install.packages("opendatatoronto"):
"installation of package 'opendatatoronto' had non-zero exit status"

ERROR: Error in library(opendatatoronto): there is no package called 'opendatatoronto'


In [3]:
############################### Data Cleansing ##################################################
### Start cleaning up with change data type, filled in any empty data in Operating Name
biz_data$Category = as.factor(biz_data$Category)
biz_data$`Licence Address Line 2` = as.factor(biz_data$`Licence Address Line 2`)
biz_data$Issued = as.Date(biz_data$Issued)
biz_data$`Cancel Date` = as.Date(biz_data$`Cancel Date`)
biz_data$`Last Record Update` = as.Date(biz_data$`Last Record Update`)
biz_data$'Operating Name' <-ifelse(biz_data$'Operating Name' == "",biz_data$'Client Name',biz_data$'Operating Name')
#str(biz_data)
#summary(biz_data)
head(biz_data)

### remove biz that have cancel date as it indicates that business has already been cancelled (i.e. terminated)
biz_data = biz_data[which(is.na(biz_data$'Cancel Date')),]

#################################################################################################
### only select business in Toronto with zip code starting with M3, M4, M5 and M6
biz_data = biz_data[which(biz_data$'Licence Address Line 2' == "TORONTO, ON"),]
biz_data = biz_data[which(biz_data$'Licence Address Line 3' > "M3" & biz_data$'Licence Address Line 3' < "M7"),]
#################################################################################################
#biz_data = biz_data %>%
#  select(-'Cancel Date')

###make column to identify street name from Licence.Address.Line.1 (might be useful for cluster work later)
###first extracting the address number. grab only the data that are after the first space
biz_data$Street <- sub("^\\S+\\s+",'', biz_data$'Licence Address Line 1')
###first extracting the address number. remove number
biz_data$Street <- gsub("[[:digit:]]", "", biz_data$Street)
###remove other unneccessary characters
biz_data$Street <- sub('-  ', '', biz_data$Street)
biz_data$Street <- sub('&A ', '', biz_data$Street)
biz_data$Street <- sub('& A ', '', biz_data$Street)
biz_data$Street <- sub('&B ', '', biz_data$Street)
biz_data$Street <- sub('& B ', '', biz_data$Street)
biz_data$Street <- sub('&  ', '', biz_data$Street)
biz_data$Street <- sub('&& ', '', biz_data$Street)
biz_data$Street <- sub('& ', '', biz_data$Street)
biz_data$Street <- sub('/ ', '', biz_data$Street)
biz_data$Street <- sub('-A ', '', biz_data$Street)
biz_data$Street <- sub('A-', '', biz_data$Street)
biz_data$Street <- sub('- A ', '', biz_data$Street)
biz_data$Street <- sub('-B ', '', biz_data$Street)
biz_data$Street <- sub('- B ', '', biz_data$Street)
biz_data$Street <- sub("(?<=^.{0}),", "", biz_data$Street, perl = TRUE )
biz_data$Street <- sub("(?<=^.{0})&", "", biz_data$Street, perl = TRUE )
biz_data$Street <- sub("(?<=^.{0})/", "", biz_data$Street, perl = TRUE )
biz_data$Street <- sub("(?<=^.{0})-", "", biz_data$Street, perl = TRUE )
### as data is not clean, there are sometimes suite number or other suffix follow them. therefore, we clean that part up too.
biz_data$Street <- sub(", #.*", '', biz_data$Street)
biz_data$Street <- sub(" #.*", '', biz_data$Street)
biz_data$Street <- sub("#.*", '', biz_data$Street)
biz_data$Street <- sub(" ,.*", '', biz_data$Street)
biz_data$Street <- sub(",.*", '', biz_data$Street)

### use similar code to clean data and prepare for geo location
biz_data$StreetAddress = paste(biz_data$'Licence Address Line 1')
###remove other characters from address one
biz_data$StreetAddress <- sub('-  ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('&A ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('& A ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('&B ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('& B ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('&  ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('&& ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('& ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('/ ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('-A ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('A-', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('- A ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('-B ', '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub('- B ', '', biz_data$StreetAddress)
### Removing suite number or other suffix follow them. therefore
biz_data$StreetAddress <- sub(", #.*", '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub(" #.*", '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub("#.*", '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub(" ,.*", '', biz_data$StreetAddress)
biz_data$StreetAddress <- sub(",.*", '', biz_data$StreetAddress)
############################### End of Data Cleansing ###########################################


ERROR: Error in is.factor(x): object 'biz_data' not found


In [4]:
#######################Converting Address to Lat-Long ###########################################
###convert to Lat- Long using tidygeocode : Reference https://www.youtube.com/watch?v=7nFZ5BwkAXc
### Load libraries needed for geocode
if (!require(tidyverse)) install.packages("tidyverse")
library(tidyverse)  
if (!require(tidygeocoder)) install.packages("tidygeocoder")
library(tidygeocoder)  
if (!require(sf)) install.packages("sf")
library(sf) 
if (!require(mapview)) install.packages("mapview")
library(mapview) 

### Prepare Data to be converted
biz_data$Address <- paste(biz_data$StreetAddress,biz_data$'Licence Address Line 2',biz_data$'Licence Address Line 3', sep = ",")
biz_geo_loc = biz_data %>%
  select(Category,`Operating Name`,city = 'Licence Address Line 2',Zip = 'Licence Address Line 3',Street, Address)

### We narrow down the area to save time in processing, we have identify Toronoto City Center with following zipcode
###area around M5C = M5H(W), M5B(N), M5E(S), M5A (E), M5G (NW), M5J (SW), M5T & M5V as extended area
### for test purpose, select only M5C, M5E

#biz_geo_loc = biz_geo_loc %>% filter(str_detect(Zip, "M5C|M5E|M5B|M5G|M5H|M5T|M5V|M5J|M5A"))
#biz_geo_loc = biz_geo_loc %>% filter(str_detect(Zip, "M5C|M5E"))

biz_geo_loc = biz_geo_loc %>% filter(str_detect(Zip, "M6G|M6H|M6J|M5T|M5S|M5R|M4V"))

### Convert address to lat - long
biz_geo_loc = biz_geo_loc %>%
  tidygeocoder:: geocode(address = Address, method = "osm")

###check if any na
sapply(biz_geo_loc, function(x) sum(is.na(x)))

biz_geo_loc1 = biz_geo_loc[-which(is.na(biz_geo_loc$lat)),]
biz_geo_loc2 = biz_geo_loc[which(is.na(biz_geo_loc$lat)),]


###display on map
biz_geo_loc_sf = biz_geo_loc1 %>%
  st_as_sf(
    coords = c("long","lat"),
    crs = 4326
  )

biz_geo_loc_sf %>% mapview()


Loading required package: tidyverse
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v tibble  2.1.1     v purrr   0.3.2
v readr   1.3.1     v stringr 1.4.0
v tibble  2.1.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: tidygeocoder
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'tidygeocoder'"also installing the dependencies 'cli', 'rlang', 'lifecycle'

Warning message:
"unable to access index for repository https://cran.r-project.org/bin/windows/contrib/3.6:
  cannot open URL 'https://cran.r-project.org/bin/windows/contrib/3.6/PACKAGES'"Packages which are only available in source form, and may need
  compilation of C/C++/F

  These will not be installed


installing the source packages 'lifecycle', 'tidygeocoder'

Warning message in install.packages("tidygeocoder"):
"installation of package 'lifecycle' had non-zero exit status"Warning message in install.packages("tidygeocoder"):
"installation of package 'tidygeocoder' had non-zero exit status"

ERROR: Error in library(tidygeocoder): there is no package called 'tidygeocoder'
